In [10]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-7o2trf2z
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-7o2trf2z
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=30c6219e042d60547292af2a765b4a1ed57c98544bc415ebebc20ebe8700cb95
  Stored in directory: /tmp/pip-ephem-wheel-cache-8lexkz4j/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [11]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [15]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void patternPrint(char *k_input,char *k_output,int len){
    int id =threadIdx.x;
    int s1 =(int)(id*(id-1)/2);
    if(s1<0){
        s1=0;
    }
    int st = id*len - s1 ;
    int l = len - id;
    for(int i=0;i<l;i++){
        k_output[st+i] = k_input[i];
    }
}

int main(){
    int len=4;
    char input[4]={'P','C','A','P'};
    int outputlength =(int)(len*(len+1)/2);
    char output[outputlength];
    printf("Input string is :%s\n",input);
    char *k_input ,*k_output;

    cudaMalloc((void**)(&k_input),sizeof(input));
    cudaMalloc((void**)(&k_output),sizeof(output));
    cudaMemcpy(k_input,input,sizeof(input),cudaMemcpyHostToDevice);
    patternPrint<<<1,len>>>(k_input,k_output,len);
    cudaError t  = cudaMemcpy(output,k_output,sizeof(output),cudaMemcpyDeviceToHost);
    if(t!= cudaSuccess){
        printf("Error : %s\n",cudaGetErrorString(t));
    }
    printf("result : %s\n",output);
    cudaFree(k_input);
    cudaFree(k_output);
   
    return 0;
}




Input string is :PCAP
result : PCAPPCAPCP

